<a href="https://colab.research.google.com/github/andersknudby/Teledetection/blob/master/Chapitre_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapitre 7 - Travailler avec des données vectorielles
Ce chapitre vous aidera à apprendre comment lire et écrire des données vectorielles, et comment travailler avec des données vectorielles et des données d'image en combinaison - plus précisément, comment extraire des valeurs d'une image matricielle pour des emplacements dans le fichier vectoriel. Le travail avec des données vectorielles en Python est, d'une certaine manière, similaire au travail avec des données matricielles - GDAL fournit une structure de base qui inclut un jeu de données avec des propriétés, et le jeu de données peut inclure un certain nombre de couches, les couches contiennent des caractéristiques, et les caractéristiques ont des attributs (comme ceux que vous voyez dans le tableau d'attributs pour n'importe quel fichier de forme ('shapefile') lorsque vous l'ouvrez dans un logiciel SIG). En plus de GDAL, plusieurs autres bibliothèques Python ont été créées pour faciliter et accélérer la manipulation des données vectorielles. Pour commencer, nous devons installer rasterio, ainsi qu'une bibliothèque appelée geopandas.

In [ ]:
!pip install rasterio
!pip install geopandas

from shapely.geometry import Point
import rasterio
import geopandas as gpd

Comme d'habitude, nous devons également donner à Colab l'accès à certains fichiers de notre Google Drive :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

myDir = '/content/drive/My Drive/Python files/'

import os
if os.path.exists(myDir + 'sfu.tif'):
  print("Drive mounted and directory found")
else:
  print("No access to the files")

Mounted at /content/drive
Drive mounted and directory found


Ok, c'était la configuration. Sur Google Drive, dans le dossier Python files, il y a un fichier appelé 'points.shp'. Si vous l'ouvrez dans QGIS, vous verrez qu'il n'y a que six points, et si vous les superposez à l'image 'sfu.tif', vous verrez qu'ils se situent tous à l'intérieur de cette image, et que les deux fichiers ont le même système de référence de coordonnées, donc les superposer est simple :

In [ ]:
pointsFilename = myDir + "points.shp"
pts = gpd.read_file(pointsFilename)  # I often use 'pts' as short for 'points' This is fairly common.
pts

,LandCover,Altitude,geometry
0,Forest,287,POINT (506722.856 5458623.040)
1,Forest,275,POINT (506902.335 5458647.466)
2,Forest,280,POINT (506706.926 5458652.776)
3,Parking lot,290,POINT (506693.651 5458566.223)
4,Parking lot,290,POINT (506707.988 5458563.037)
5,Parking lot,291,POINT (506728.166 5458552.417)


Facile, non ? La variable 'pts' est un objet appelé un **geodataframe**, qui correspond assez fidèlement au tableau d'attributs du fichier de forme - chaque point est stocké comme une 'caractéristique' dans une ligne, et chaque attribut est stocké dans une colonne.

Un **géodataframe** est basé sur un plus petit type d'objet de la bibliothèque pandas, appelé **dataframe**. Mais la particularité d'un géodataframe est qu'il est toujours associé à des informations géographiques, dans l'attribut "geometry". Dans l'ensemble de données 'pts', vous verrez que ces informations contiennent l'étiquette 'POINT', ainsi que les coordonnées x et y de chaque point.

Nous pouvons obtenir quelques informations de base sur ce cadre de données géographiques :

In [ ]:
print("Number of rows:", len(pts))
pts.crs

Number of rows: 7


<Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Vous pouvez travailler avec les géodonnées de plusieurs façons ; nous allons nous contenter d'examiner les méthodes les plus courantes pour les manipuler.

Si vous souhaitez supprimer une colonne, vous pouvez utiliser la fonction 'drop' :

In [ ]:
pts = pts.drop(['LandCover'], axis=1)
pts

,Altitude,geometry
0,287,POINT (506722.856 5458623.040)
1,275,POINT (506902.335 5458647.466)
2,280,POINT (506706.926 5458652.776)
3,290,POINT (506693.651 5458566.223)
4,290,POINT (506707.988 5458563.037)
5,291,POINT (506728.166 5458552.417)


Et si vous voulez ajouter une colonne, vous pouvez le faire comme ceci :

In [ ]:
owner = ['SFU', 'SFU', 'SFU', 'SFU', 'SFU', 'SFU']
pts['Owner'] = owner
pts

,Altitude,geometry,Owner
0,287,POINT (506722.856 5458623.040),SFU
1,275,POINT (506902.335 5458647.466),SFU
2,280,POINT (506706.926 5458652.776),SFU
3,290,POINT (506693.651 5458566.223),SFU
4,290,POINT (506707.988 5458563.037),SFU
5,291,POINT (506728.166 5458552.417),SFU


De même, si vous voulez supprimer la première ligne :

In [ ]:
pts = pts.drop([0])
pts

,Altitude,geometry,Owner
1,275,POINT (506902.335 5458647.466),SFU
2,280,POINT (506706.926 5458652.776),SFU
3,290,POINT (506693.651 5458566.223),SFU
4,290,POINT (506707.988 5458563.037),SFU
5,291,POINT (506728.166 5458552.417),SFU


Si vous voulez ajouter une nouvelle ligne (ou plusieurs lignes), cela devient un peu plus compliqué, mais pas impossible :

In [ ]:
toAdd = [{'Altitude': 567, 'Owner': 'SFU', 'geometry': Point(506700, 5458600)},
         {'Altitude': 234, 'Owner': 'SFU', 'geometry': Point(506696, 5458612)}]

pts = pts.append(gpd.GeoDataFrame(toAdd))
pts

,Altitude,geometry,Owner
1,275,POINT (506902.335 5458647.466),SFU
2,280,POINT (506706.926 5458652.776),SFU
3,290,POINT (506693.651 5458566.223),SFU
4,290,POINT (506707.988 5458563.037),SFU
5,291,POINT (506728.166 5458552.417),SFU
0,567,POINT (506700.000 5458600.000),SFU
1,234,POINT (506696.000 5458612.000),SFU


## Extraire des valeurs d'un fichier matriciel vers un fichier de forme de points
Il est très fréquent que vous ayez un ensemble de points, peut-être des endroits où vous avez été et observé quelque chose, et vous voulez extraire des valeurs d'un fichier raster exactement à ces points. Il existe des outils dans ArcGIS et QGIS pour faire cela, mais c'est également très facile à faire avec Python et geopandas. Tout d'abord, nous devons ouvrir le fichier image, comme nous l'avons fait dans le dernier chapitre :

In [ ]:
imageFilename = myDir + 'sfu.tif'
ds = rasterio.open(imageFilename)  # ds is a commonly used shorthand for 'dataset'

Nous devons ensuite obtenir toutes les coordonnées (x, y) du fichier de forme des points. Nous pouvons le faire dans une boucle 'for', où nous utilisons 'i' pour itérer à travers les numéros de ligne dans 'pts'. Pour ce faire, nous utilisons la fonction 'iloc', qui renvoie le numéro de ligne donné à partir du cadre de géodonnées :

In [ ]:
coords = []
for i in range(len(pts)):
  x = pts.iloc[i].geometry.x
  y = pts.iloc[i].geometry.y
  coords.append((x, y))
coords

[(506902.3346581877, 5458647.466136725),
 (506706.926073132, 5458652.776152623),
 (506693.6510333864, 5458566.222893482),
 (506707.98807631165, 5458563.036883943),
 (506728.16613672505, 5458552.416852146),
 (506700.0, 5458600.0),
 (506696.0, 5458612.0)]

Il y a souvent d'autres façons de faire des choses itératives en Python qu'avec une boucle for. Une alternative à ce qui précède consiste à utiliser "list comprehension", dont le code est plus complexe à écrire, mais qui nécessite moins de lignes et s'exécute plus rapidement. Le résultat est exactement le même :

In [ ]:
coords = [(x, y) for x, y in zip(pts.geometry.x, pts.geometry.y)]
coords

[(506902.3346581877, 5458647.466136725),
 (506706.926073132, 5458652.776152623),
 (506693.6510333864, 5458566.222893482),
 (506707.98807631165, 5458563.036883943),
 (506728.16613672505, 5458552.416852146),
 (506700.0, 5458600.0),
 (506696.0, 5458612.0)]

Lorsque nous avons les coordonnées de tous les points, nous pouvons utiliser la fonction 'sample' de rasterio pour obtenir les valeurs des pixels - dans chaque bande, pour chacun des emplacements des points. Pour cela, nous utilisons une autre liste de compréhension. Rappelez-vous que dans notre image, les bandes rouges étaient les premières, puis les vertes, et enfin les bleues. Nous devons fournir cette information comme argument 'indexes' à la fonction sample.

In [ ]:
pts["red"] = [x[0] for x in ds.sample(coords, indexes=1)]
pts["green"] = [x[0] for x in ds.sample(coords, indexes=2)]
pts["blue"] = [x[0] for x in ds.sample(coords, indexes=3)]
pts

,Altitude,geometry,Owner,red,green,blue
1,275,POINT (506902.335 5458647.466),SFU,46,66,54
2,280,POINT (506706.926 5458652.776),SFU,27,29,18
3,290,POINT (506693.651 5458566.223),SFU,147,166,172
4,290,POINT (506707.988 5458563.037),SFU,149,168,172
5,291,POINT (506728.166 5458552.417),SFU,141,160,166
0,567,POINT (506700.000 5458600.000),SFU,88,122,145
1,234,POINT (506696.000 5458612.000),SFU,41,51,39


Cela fonctionne bien car nos deux systèmes de coordonnées - pour les données matricielles et vectorielles - sont alignés. Mais que se passe-t-il si ce n'est pas le cas ? Dans ce cas, vous devez d'abord reprojeter l'un des ensembles de données pour qu'il corresponde aux coordonnées de l'autre. En général, le plus simple, et certainement le plus rapide, est de reprojeter les données vectorielles, car le nombre de calculs nécessaires pour reprojeter les données matricielles est beaucoup plus important. Faisons un essai, même si nous n'en avons pas besoin avec ces deux ensembles de données.

La première étape consiste à tester réellement si les deux systèmes de référence de coordonnées sont les mêmes. Cela peut être fait comme suit :

In [ ]:
epsgPoints = pts.crs.to_epsg()  # Get the EPSG number of the CRS of the vector data
epsgRaster = ds.crs.to_epsg()  # Get the EPSG number of the CRS of the vector data
epsgPoints == epsgRaster  # This tests whether the two EPSG numbers are the same.

True

Mais bon, imaginons que les données matricielles utilisent plutôt EPSG 32609 (c'est-à-dire une zone UTM à l'ouest). Nous devrions alors reprojeter les points pour utiliser les coordonnées de cette zone UTM à la place :

In [ ]:
newPts = pts.to_crs("EPSG:32609")

Maintenant, nous allons essayer de répéter ce que nous avons fait ci-dessus, mais avec les données reprojetées. Je sais que c'est faux, mais ça peut être intéressant de voir ce qui se passe :

In [ ]:
coords = [(x, y) for x, y in zip(newPts.geometry.x, newPts.geometry.y)]
pts["red"] = [x[0] for x in ds.sample(coords, indexes=1)]
pts["green"] = [x[0] for x in ds.sample(coords, indexes=2)]
pts["blue"] = [x[0] for x in ds.sample(coords, indexes=3)]
pts

,Altitude,geometry,Owner,red,green,blue
1,275,POINT (506902.335 5458647.466),SFU,NaN,NaN,NaN
2,280,POINT (506706.926 5458652.776),SFU,NaN,NaN,NaN
3,290,POINT (506693.651 5458566.223),SFU,NaN,NaN,NaN
4,290,POINT (506707.988 5458563.037),SFU,NaN,NaN,NaN
5,291,POINT (506728.166 5458552.417),SFU,NaN,NaN,NaN
0,567,POINT (506700.000 5458600.000),SFU,NaN,NaN,NaN
1,234,POINT (506696.000 5458612.000),SFU,NaN,NaN,NaN


C'est un autre exemple d'erreur sémantique, et une erreur difficile à trouver. Python pense que tout s'est bien passé, et toutes les valeurs NaN que nous voyons maintenant dans newPts indiquent que l'image n'a aucune valeur associée aux emplacements des points. Mais c'est parce que les 'coords' font maintenant référence à un CRS différent des coordonnées de l'image, et la fonction 'sample' n'a aucun moyen de le savoir.

**Leçon importante :** C'est à vous, lorsque vous écrivez votre code, de vérifier si les systèmes de référence des coordonnées de vos données correspondent ! Si ce n'est pas le cas, Python n'affichera peut-être pas d'erreur, mais votre code sera peut-être erroné !

La dernière chose que nous voulons faire est de réécrire le geodataframe dans un shapefile - étant donné que nous venons de faire quelque chose de mal, cela ne sera utile que comme exercice :

In [ ]:
newPtsFilename = myDir + "newPoints.shp"
newPts.to_file(newPtsFilename)

Et bien sûr, nous devons démonter et vider les fichiers pour les transférer sur Google Drive :

In [ ]:
drive.flush_and_unmount()

##Exercice
Dans un nouveau cahier, écrivez le code qui fait ce qui suit :

1\) Lecture de l'image 'sfu.tif'.<br>
2\) Calculer le CCV de l'image (voir exercice du chapitre précédent).<br>
3\) Lire le fichier 'points.shp'. <br>
4\) Extraire la valeur du CCV pour chaque point.<br>
5\) Écrire les résultats sous forme d'un nouveau fichier shapefile.<br>
6\) Ouvrez l'image originale et le nouveau fichier de forme dans QGIS, et vérifiez si les points ayant les grandes valeurs CCV sont ceux situés au-dessus de la végétation.<br>


## Extraire des valeurs d'un raster vers un fichier de forme polygonal 
Il est parfois plus utile de travailler avec des polygones qu'avec des points, et nous pouvons également utiliser Python pour extraire des valeurs à partir de données matricielles vers des fichiers de forme polygonaux. Généralement, cette opération est souvent appelée "statistiques zonales". Pour ce faire, nous allons installer et utiliser une autre bibliothèque, appelée "rasterstats" :

In [ ]:
!pip install rasterstats
import rasterstats

     |████████████████████████████████| 133kB 5.9MB/s 


Pour le fichier raster, nous pouvons travailler avec l'image 'sfu.tif' qui est déjà chargée. Et pour les données polygonales, nous pouvons travailler avec le nom de fichier 'polygons.shp' :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

myDir = '/content/drive/My Drive/Python files/'

import os
if os.path.exists(myDir + 'sfu.tif'):
  print("Drive mounted and directory found")
else:
  print("No access to the files")

polygonsFilename = myDir + "polygons.shp"
polys = gpd.read_file(polygonsFilename)  # I often use 'pts' as short for 'points' This is fairly common.
polys

Mounted at /content/drive
Drive mounted and directory found


,Type,geometry
0,Soccer field,"POLYGON ((506671.880 5458456.306, 506762.150 5..."
1,Roof,"POLYGON ((506827.994 5458444.093, 506869.413 5..."
2,Forest,"POLYGON ((506963.931 5458670.299, 506975.613 5..."


Essayons de trouver lequel des trois polygones est le plus lumineux. Pour ce faire, nous devons d'abord calculer la luminosité de l'image :

In [ ]:
import numpy as np  # We need NumPy for this

# And we need to open the image file again
imageFilename = myDir + 'sfu.tif'
ds = rasterio.open(imageFilename)  # ds is a commonly used shorthand for 'dataset'

band1 = ds.read(1).astype('uint16')
band2 = ds.read(2).astype('uint16')
band3 = ds.read(3).astype('uint16')
brightness = (band1 + band2 + band3) / 3
brightness

array([[ 43.33333333,  49.33333333,  54.        , ...,  57.66666667,
         58.66666667,  56.33333333],
       [ 34.66666667,  35.66666667,  47.        , ...,  57.33333333,
         53.66666667,  52.        ],
       [ 41.66666667,  38.66666667,  40.33333333, ...,  45.        ,
         49.        ,  53.        ],
       ...,
       [ 88.66666667,  90.        , 105.        , ...,  28.        ,
         26.33333333,  25.        ],
       [ 70.66666667,  76.        ,  99.        , ...,  26.33333333,
         24.33333333,  24.        ],
       [ 57.66666667,  70.33333333, 100.        , ...,  25.        ,
         23.66666667,  24.33333333]])

Maintenant, la luminosité est un tableau NumPy, mais il reste deux problèmes :

1\) Les tableaux NumPy ne sont pas associés à des informations de localisation, nous devons donc extraire ces informations des données matricielles. Sinon, il n'y a aucun moyen d'associer les coordonnées de chaque polygone aux lignes et colonnes du tableau NumPy ! Pour ce faire, nous pouvons utiliser les informations de géotransformation de ds, en utilisant 'ds.transform'.

2\) Nous devons également fournir des informations sur la valeur, dans le tableau NumPy, qui est utilisée pour les données manquantes. Il s'agit de la valeur standard "nan" utilisée par NumPy.

Au final, nous pouvons utiliser la fonction rasterstats zonal_stats comme suit :


In [ ]:
stats = rasterstats.zonal_stats(polys, band1, nodata=np.nan, affine=ds.transform)  # This works
stats

[{'count': 480124, 'max': 251.0, 'mean': 140.33809599186876, 'min': 42.0},
 {'count': 62428, 'max': 237.0, 'mean': 74.28887678605754, 'min': 9.0},
 {'count': 93533, 'max': 100.0, 'mean': 35.84866303871361, 'min': 7.0}]

Notez que la fonction zonal_stats peut également travailler avec des fichiers directement, plutôt qu'avec des objets de données déjà lus dans Python. Voici deux exemples qui donneraient les mêmes résultats que le précédent :

In [ ]:
stats = rasterstats.zonal_stats(polygonsFilename, imageFilename)  # Using files for both vector and raster data
stats = rasterstats.zonal_stats(polys, imageFilename)  # Using the 'polys' object with the raster file

##Exercice
Trouvez lequel des trois polygones a la valeur moyenne de CCV la plus élevée.